In [1]:
import os
import glob
import numpy as np
import pprint
import functools

import tensorflow as tf

from ipywidgets import interact, interactive, fixed, interact_manual, HBox, VBox, Button
import ipywidgets as widgets

/home/lcances/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/lcances/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/lcances/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/lcances/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
import sys
sys.executable

'/home/lcances/.miniconda3/envs/tensorboard/bin/python'

In [3]:
# global variables
tag_list = []
log_list = []
file_stats = dict()
stat_list = ["mean", "std", "maxi", "mini"]

os.chdir("../..")
tensorboard_root = "tensorboard"

# Utility function

In [4]:
def get_all_tag(path):
    global tag_list
    tag_list = []

    for e in tf.compat.v1.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag not in tag_list:
                    tag_list.append(value.tag)
    
    return tag_list

- Si la list n'est pas vide
    - Si le premier element est un fichier: return True
    - Si le premier element est un dossier:
        - lister le contenue de ce dossier L
        - si L n'est pas vide
            - Si le premier element est un fichier: return True
            - Sinon return False

In [5]:
def contain_log(path):
    if path:
        if os.path.isfile(path[0]):
            return True
        
        if os.path.isdir(path[0]):
            if not os.listdir(path[0]): # directory is empty
                return False
            
            sublist = [os.path.join(path[0], p) for p in os.listdir(path[0])]
            
            if sublist:
                if os.path.isfile(sublist[0]):
                    return True
                else:
                    return False
    return True

In [6]:
def get_tag_list(log_file):
    list_file = os.listdir(log_file)
    
    print(log_file)
    print(list_file)
    path = os.path.join(log_file, list_file[0])
    tag_list = get_all_tag(path)
    print(tag_list)

In [7]:
tensorboard_root

'tensorboard'

In [8]:
!pwd

/home/lcances/sync/Documents_sync/Projet/Deep-Co-Training


In [9]:
glob.glob(tensorboard_root + "/**")

['tensorboard/ubs8k']

In [10]:
def ui_dropdown_dir(directory):
    global log_list
    
    sub_directories = sorted([os.path.join(directory, p) for p in os.listdir(directory) ])
        
    if not contain_log(sub_directories):
        interact(ui_dropdown_dir, directory=sub_directories)
        
    else:
        path_list = [os.path.join(directory, l) for l in os.listdir(directory)]
        log_list = path_list
        interact_manual(get_tag_list, log_file=path_list)

In [11]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"))

interactive(children=(Dropdown(description='directory', options=('tensorboard/ubs8k',), value='tensorboard/ubs…

<function __main__.ui_dropdown_dir(directory)>

In [12]:
!ls ../../tensorboard/ubs8k

ls: cannot access '../../tensorboard/ubs8k': No such file or directory


In [13]:
def stat_stat(stat = "maxi"):
    global file_stats
    
    values = [file_stats[l][stat] for l in file_stats.keys()]
    
    print("stat stat")
    print("mean: ", np.nanmean(values))
    print("std : ", np.nanstd(values))
    print("mini: ", np.nanmin(values))
    print("maxi: ", np.nanmax(values))
    print("")
    print("%.3f ± %.3f" % (np.nanmean(values), np.nanstd(values)))
    print("")
    print("detail ")
    pprint.pprint(values)

In [14]:
glob.glob(tensorboard_root + "/**")

['tensorboard/ubs8k']

In [15]:
def ui_dropdown_dir(directory, filters: str = ""):
    global log_list
    
    sub_directories = sorted([os.path.join(directory, p) for p in os.listdir(directory) ])
        
    if not contain_log(sub_directories):
        print("next")
        ui = interact(ui_dropdown_dir, directory=sub_directories, filters="")
        
    else:
        path_list = [os.path.join(directory, l) for l in os.listdir(directory) if filters in l]
        log_list = path_list
        interact_manual(get_tag_list, log_file=path_list)

In [35]:
ui = interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"),  filters="")

interactive(children=(Dropdown(description='directory', options=('tensorboard/ubs8k',), value='tensorboard/ubs…

In [17]:
!ls ../../tensorboard/ubs8k

ls: cannot access '../../tensorboard/ubs8k': No such file or directory


In [18]:
def stat_stat(stat = "maxi"):
    global file_stats
    
    values = [file_stats[l][stat] for l in file_stats.keys()]
    
    print("stat stat")
    print("mean: ", np.nanmean(values))
    print("std : ", np.nanstd(values))
    print("mini: ", np.nanmin(values))
    print("maxi: ", np.nanmax(values))
    print("")
    print("%.3f ± %.3f" % (np.nanmean(values) * 100, np.nanstd(values) * 100))
    print("")
    print("detail ")
    pprint.pprint(values)

In [19]:
def format_logs(stat = "maxi", format_fn = None):
    global file_stats
    
    msg = ""
    for k, d in file_stats.items():
        formated_name = k
        
        if format_fn is not None:
            formated_name = format_fn(k)
            
        msg += "%s %s\n" % (formated_name, d[stat])
    return msg
            
    

In [24]:
def statistics(mean=True, std=True, max=False, min=False, tag=tag_list[0]):
    global file_stats
    
    file_log = dict()
    
    # read all the log files ------------------------------
    @functools.lru_cache()
    def get_log_data(path):
        logs = dict()

        for e in tf.compat.v1.train.summary_iterator(path):
            for value in e.summary.value:
                if value.HasField("simple_value"):
                    if value.tag not in logs:
                        logs[value.tag] = []
                    logs[value.tag].append(value.simple_value)
    
        return logs
    
    for l in log_list:
        final_path = os.path.join(l, os.listdir(l)[0])
        logs = get_log_data(final_path)
        
        if logs:
            file_log[l] = logs
        
    # compute the statistics ------------------------------
    # ---- file wise ----
    file_stats = dict()
    
    for l in file_log.keys():
        file_stats[l] = dict(
            mean = np.nanmean(file_log[l][tag]),
            std = np.nanstd(file_log[l][tag]),
            mini = np.nanmin(file_log[l][tag]),
            maxi = np.nanmax(file_log[l][tag]),
        )
        
    interact(stat_stat, stat=stat_list)

In [36]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"), filters="")

interactive(children=(Dropdown(description='directory', options=('tensorboard/ubs8k',), value='tensorboard/ubs…

<function __main__.ui_dropdown_dir(directory, filters:str='')>

In [37]:
interact_manual(statistics, mean=True, std=True, mini=False, maxi=False, tag=tag_list)

interactive(children=(Checkbox(value=True, description='mean'), Checkbox(value=True, description='std'), Check…

<function __main__.statistics(mean=True, std=True, max=False, min=False, tag='train/total_loss')>

In [34]:
def format_fn(n):
    base = n.split("/")[4].split("_")
    print(base)
    model_name = base[2]
    epoch = base[4]
    lr = base[5]
    rule = base[6]
    nb_steps = base[-2][:-1]
    
    formated = "%s %s %s %s" % (model_name, lr, rule, nb_steps)
    return formated

detail = format_logs("maxi", format_fn).split("\n")
detail = sorted(detail, key=lambda x: x.split(" ")[0])
print("\n".join(detail))

['2020-08-10', '14:32:01', 'cnn0', '0.1S', '1000e', '0.0005lr', 'linear-rfn', '5s', '']
['2020-08-10', '16:01:35', 'cnn03', '0.1S', '1000e', '0.0005lr', 'weighted-sigmoid-rfn', '10s', '']
['2020-08-10', '14:47:12', 'cnn0', '0.1S', '1000e', '0.0005lr', 'sigmoid-rfn', '10s', '']
['2020-08-10', '15:23:38', 'cnn03', '0.1S', '1000e', '0.0005lr', 'linear-rfn', '10s', '']
['2020-08-10', '14:50:10', 'cnn0', '0.1S', '1000e', '0.003lr', 'weighted-linear-rfn', '5s', '']
['2020-08-10', '15:17:04', 'cnn03', '0.1S', '1000e', '0.003lr', 'linear-rfn', '10s', '']
['2020-08-10', '14:41:10', 'cnn0', '0.1S', '1000e', '0.003lr', 'sigmoid-rfn', '10s', '']
['2020-08-10', '15:48:52', 'cnn03', '0.1S', '1000e', '0.0005lr', 'weighted-linear-rfn', '10s', '']
['2020-08-10', '15:58:24', 'cnn03', '0.1S', '1000e', '0.0005lr', 'weighted-sigmoid-rfn', '5s', '']
['2020-08-10', '15:36:26', 'cnn03', '0.1S', '1000e', '0.0005lr', 'sigmoid-rfn', '10s', '']
['2020-08-10', '15:45:47', 'cnn03', '0.1S', '1000e', '0.0005lr', 'wei

In [49]:
def contain_log(path):
    if path:
        
        if os.path.isfile(path[0]):
            return True
        
        if os.path.isdir(path[0]):
            sublist = [os.path.join(path[0], p) for p in os.listdir(path[0])]
            
            if sublist:
                if os.path.isfile(sublist[0]):
                    return True
                else:
                    return False
    return True

In [50]:
root = "/mnt/e/sync/Documents_sync/Projet/UrbanSound8K/tensorboard/osirim_tensorboard"
list_dir = os.listdir(root)

for name in list_dir:
    path = os.path.join(root, name)
    
    print(contain_log(path))

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/e/sync/Documents_sync/Projet/UrbanSound8K/tensorboard/osirim_tensorboard'

In [51]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"))

interactive(children=(Dropdown(description='directory', options=(), value=None), Output()), _dom_classes=('wid…

<function __main__.ui_dropdown_dir(directory)>